## Import libraries

In [1]:
import dash # initialize your application
import dash_core_components as dcc # allows you to create interactive components like graphs, dropdowns, or date ranges
import dash_html_components as html # access HTML tags
import pandas as pd
import numpy as np
import plotly.graph_objects as go

from datetime import datetime
from dash.dependencies import Output, Input

## Import data

In [2]:
data = pd.read_csv("Swim_dataset.csv") # read csv file

# filter the unnecessary columns
unnec_cols = data.columns[10:]
data = data.drop(columns=unnec_cols)

# rename "Unnamed: 0" column name to "Date"
data = data.rename(columns={"Unnamed: 0":"Date"})

# convert the contents in the date column into a datetime format
data["Date"] = pd.to_datetime(data["Date"], format="%Y-%m-%d") # convert the date into a date format

# set Date as an index 
# data = data.set_index('Date')

# convert nan to 0
data = data.fillna(0)

# remove all zeros
column_list = data.columns[1:]
data = data[(data[column_list].T != 0.0).any()]

data.head()

,Date,Warm Up,Kick,Pull,Swim,Drill,Main Set,Post Set,Cool Down,Total
0,2020-01-02,400.0,600.0,0.0,600.0,400.0,2800.0,100.0,400.0,5300
1,2020-01-03,400.0,0.0,0.0,1600.0,400.0,2000.0,600.0,400.0,5400
4,2020-01-06,800.0,0.0,0.0,2200.0,0.0,2500.0,880.0,200.0,6580
5,2020-01-07,800.0,0.0,0.0,920.0,0.0,3000.0,0.0,400.0,5120
6,2020-01-08,800.0,0.0,0.0,1000.0,0.0,4500.0,0.0,400.0,6700


In [3]:
# 1일치
one_day_data = data.iloc[0][1:-1]

# columns
categories = data.columns[1:-1]

In [4]:
def convertToSec(s):
    if s != 0:
        minute = int(s[:2])
        second = int(s[3:5])
        millisecond = int(s[6:])
        return minute*60 + second + millisecond/100
    return 0

In [5]:
competition_data = pd.read_csv("Swim_progress.csv") # read csv file

# fill NaN with zeros
competition_data = competition_data.fillna(0)

# convert Date column as an index
competition_data['Date'] = pd.to_datetime(competition_data['Date'])
competition_data.set_index('Date', inplace=True)

# drop competition name **나중에 필요하면 이거 지우시면 됩니다 (대신 밑에 코드 수정 필요)**
competition_data= competition_data.drop(axis=0, columns="Competition Name")

# columns names
competition_col_list = competition_data.columns

for i in range(len(competition_data)):
    for col_name in competition_col_list:
        converted_value = convertToSec(competition_data.iloc[i, competition_data.columns.get_loc(col_name)])
        competition_data.iloc[i, competition_data.columns.get_loc(col_name)] = converted_value

competition_data.head()

,50 Free,100 Free,200 Free,400 Free,800 Free,1500 Free,50 Fly,100 Fly,200 Fly,50 Back,100 Back,200 Back,50 Breast,100 Breast,200 Breast,100 IM,200 IM,400 IM
Date,,,,,,,,,,,,,,,,,,
2020-03-11,0,0,112.76,0,0.0,0.0,28.12,60.09,0.0,0,0.0,0.0,0,0,0.0,0.0,0,0.0
2020-03-18,24.19,52.16,0,0,0.0,0.0,0,0,0.0,29.2,0.0,0.0,30.77,67.04,0.0,0.0,130.51,0.0
2020-03-25,0,52.05,0,0,0.0,0.0,27.98,0,0.0,29.06,0.0,0.0,0,0,0.0,0.0,0,0.0
2020-03-30,0,53.53,116.26,0,0.0,0.0,0,0,0.0,0,0.0,0.0,0,0,0.0,0.0,0,0.0
2020-04-05,23.9,0,0,0,0.0,0.0,0,0,0.0,30.67,0.0,0.0,31.63,67.26,0.0,0.0,128.58,0.0


## Initialize the application

In [6]:
app = dash.Dash(
    __name__,
    meta_tags=[{"name": "viewport", "content": "width=device-width, initial-scale=1"}],
)
app.title = "Swimming Training Time Analysis"

## App layout

In [7]:
graph_bgcolor = "#0A165A"
font_color = "#FFFFFF"

In [8]:
app.layout = html.Div(
    children=[
        
        # heading
        html.Div(
            children=[
                html.P(
                    children="🏊‍♂️", className="header-emoji"
                ),
                html.H1(
                    children="Swimming Training Analysis", className="header-title"
                ),
                html.P(
                    children="Analyze the how the training time affects the result",
                    className="header-description",
                ),
            ],
            className="header",
        ),
        
        # Graph part
        html.Div(
            children=[
                
                # Competition graph
                html.Div(
                    children=[
                        
                        # title
                        html.Div(children='Competition Graph', className='competition-title'),
                        
                        # menu
                        html.Div(
                            children=[
                                
                                # name selector
                                dcc.Dropdown(
                                    id='competition-name-selector',
                                    options=[
                                        {'label': 'Eric Oh', 'value': 'eric'},
                                    ],
                                    value='eric',
                                    style={"min-width": "150px"},
                                ),
                                
                                # event selector
                                dcc.Dropdown(
                                    id='competition-event-selector',
                                    options=[
                                        {'label': '50 Free', 'value': '50free'},
                                        {'label': '100 Free', 'value': '100free'},
                                        {'label': '200 Free', 'value': '200free'},
                                        {'label': '400 Free', 'value': '400free'},
                                        {'label': '50 Fly', 'value': '50fly'},
                                        {'label': '100 Fly', 'value': '100fly'},
                                        {'label': '50 Back', 'value': '50back'},
                                        {'label': '100 Back', 'value': '100back'},
                                        {'label': '50 Breast', 'value': '50breast'},
                                        {'label': '100 Breast', 'value': '100breast'},
                                        {'label': '100 IM', 'value': '100im'},
                                        {'label': '200 IM', 'value': '200im'},
                                    ],
                                    value='50free',
                                    style={"min-width": "200px"},
                                ),
                                
                                # average-total selection
                                dcc.RadioItems(
                                    id='competition-avg-date-selector',
                                    options=[
                                        {'label': 'Monthly Average', 'value': 'monthly-avg'},
                                        {'label': 'To-the-date', 'value': 'to-the-date'},
                                    ],
                                    value='monthly-avg',
                                    labelStyle={
                                        'display': 'flex',
                                        'margin-bottom': '5px',
                                        'font-weight': 300,
                                        'color': '#FFFFFF',
                                    },
                                ),
                            ],
                            className='menu'
                        ),
                        
                        # graph
                        dcc.Graph(
                            id='competition-graph-id',
                            className='competition-graph',
                            figure={
                                "layout": {
                                    "paper_bgcolor": graph_bgcolor,
                                    "plot_bgcolor": graph_bgcolor,
                                    "font": {
                                        "color": "#FFFFFF"
                                    }
                                },
                            },
                        ),
                    ],
                    className='competition-graph-container'
                ),
                
                
                # Training graph
                html.Div(
                    children=[
                        
                        # categorical bar plot
                        html.Div(
                            children=[
                                # title
                                html.Div(
                                    children=[
                                        html.Div(children='Categorical Graph', className='categorical-graph-title'),
                                        dcc.DatePickerRange(
                                            id='category-date-picker',
                                            min_date_allowed=data.Date.min().date(),
                                            max_date_allowed=data.Date.max().date(),
                                            start_date=data.Date.min().date(),
                                            end_date=data.Date.max().date(),
                                            with_portal=True,
                                            style={'display': 'inline-block'},
                                            display_format='DD-MMM-YY',
                                            className='datepicker',
                                        ),
                                    ], 
                                    className='category-title-container',
                                ),

                                # menu
                                html.Div(
                                    children=[
                                        # average-total selection
                                        dcc.RadioItems(
                                            id='category-avg-total-selector',
                                            options=[
                                                {'label': 'Average', 'value': 'avg'},
                                                {'label': 'Total', 'value': 'total'},
                                            ],
                                            value='avg',
                                            labelStyle={
                                                'display': 'flex',
                                                'margin-bottom': '5px',
                                                'font-weight': 300,
                                                'color': '#FFFFFF',
                                            },
                                        ),

                                        # category selector
                                        dcc.Dropdown(
                                            id='category-category-selector',
                                            options=[
                                                {'label': 'Warm Up', 'value': 'Warm Up'},
                                                {'label': 'Kick', 'value': 'Kick'},
                                                {'label': 'Pull', 'value': 'Pull'},
                                                {'label': 'Swim', 'value': 'Swim'},
                                                {'label': 'Drill', 'value': 'Drill'},
                                                {'label': 'Main Set', 'value': 'Main Set'},
                                                {'label': 'Post Set', 'value': 'Post Set'},
                                                {'label': 'Cool Down', 'value': 'Cool Down'},
                                            ],
                                            value=["Warm Up","Swim","Main Set"],
                                            multi=True,
                                            style={"min-width": "150px"},
                                        ),
                                    ],
                                    className='train-menu',
                                ),

                                # bar plot
                                html.Div(
                                    children=dcc.Graph(
                                        id='categorical-bar-chart',
                                        style={"height": "100%"},
                                        config={"displayModeBar": True},
                                        figure={
                                            "layout": {
                                                "paper_bgcolor": graph_bgcolor,
                                                "plot_bgcolor": graph_bgcolor,
                                                "font": {
                                                    "color": "#FFFFFF"
                                                }
                                            },
                                        },
                                    ),
                                    className='bar-categorical-graph'
                                ),
                            ],
                            className='bar-categorical-graph-container'
                        ),
                        
                        # Categorical line plot
                        html.Div(
                            children=[
                                # title
                                html.Div(
                                    children=[
                                        html.Div(children='Time Series Graph', className='time-graph-title'),
                                        html.Div(
                                            children=[
                                                html.Button('D', id='daily-button', className='time-series-button', n_clicks=0),
                                                html.Button('W', id='weekly-button', className='time-series-button', n_clicks=0),
                                                html.Button('M', id='monthly-button', className='time-series-button', n_clicks=0),
                                            ],
                                        ),
                                    ], 
                                    className='time-title-container',
                                ),

                                # menu
                                html.Div(
                                    children=[
                                        # average-total selection
                                        dcc.RadioItems(
                                            id='time-avg-total-selector',
                                            options=[
                                                {'label': 'Average', 'value': 'avg'},
                                                {'label': 'Total', 'value': 'total'},
                                            ],
                                            value='avg',
                                            labelStyle={
                                                'display': 'flex',
                                                'margin-bottom': '5px',
                                                'font-weight': 300,
                                                'color': '#FFFFFF',
                                            },
                                        ),

                                        # category selector
                                        dcc.Dropdown(
                                            id='time-category-selector',
                                            options=[
                                                {'label': 'Warm Up', 'value': 'Warm Up'},
                                                {'label': 'Kick', 'value': 'Kick'},
                                                {'label': 'Pull', 'value': 'Pull'},
                                                {'label': 'Swim', 'value': 'Swim'},
                                                {'label': 'Drill', 'value': 'Drill'},
                                                {'label': 'Main Set', 'value': 'Main Set'},
                                                {'label': 'Post Set', 'value': 'Post Set'},
                                                {'label': 'Cool Down', 'value': 'Cool Down'},
                                            ],
                                            value=["Warm Up"],
                                            multi=True,
                                            style={"min-width": "150px"},
                                        ),
                                    ],
                                    className='train-menu'
                                ),

                                # line plot
                                html.Div(
                                    children=dcc.Graph(
                                        id='timeseries-line-chart',
                                        style={"height": "100%"},
                                        config={"displayModeBar": True},
                                        figure={
                                            "layout": {
                                                "paper_bgcolor": graph_bgcolor,
                                                "plot_bgcolor": graph_bgcolor,
                                                "font": {
                                                    "color": "#FFFFFF"
                                                }
                                            },
                                        },
                                    ),
                                    className='time-series-graph'
                                ),
                            ],
                            className='time-series-graph-container',
                        ),
                    ],
                    className='training-graph-container',
                ),
            ],
            className='graph-container'
        ),
        
        # space
        html.Div(style={"height": 80}),
    ],
    className='app-container'
)

In [9]:
# Monthly data
monthly_competition_data = competition_data.copy()
monthly_competition_data = monthly_competition_data.astype('float16') # should match the datatype
monthly_competition_data = monthly_competition_data.resample('1M').mean()
monthly_competition_data = monthly_competition_data.fillna(0)

# drop rows with all zeros
compeition_column_list = monthly_competition_data.columns
monthly_competition_data = monthly_competition_data[(monthly_competition_data[compeition_column_list].T != 0.0).any()]
monthly_competition_data

,50 Free,100 Free,200 Free,400 Free,800 Free,1500 Free,50 Fly,100 Fly,200 Fly,50 Back,100 Back,200 Back,50 Breast,100 Breast,200 Breast,100 IM,200 IM,400 IM
Date,,,,,,,,,,,,,,,,,,
2020-03-31,6.046875,39.43750,57.25000,0.0000,0.0,0.0,14.03125,15.023438,0.0,14.562500,0.0,0.0,7.691406,16.765625,0.0,0.0,32.6250,0.0
2020-04-30,24.046875,25.90625,56.09375,123.8125,0.0,0.0,0.00000,0.000000,0.0,15.335938,0.0,0.0,15.812500,33.625000,0.0,0.0,64.3125,0.0
2020-07-31,0.000000,53.40625,115.87500,0.0000,0.0,0.0,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0000,0.0


# 50 Free

In [10]:
_50free_data = competition_data['50 Free'].to_frame()
_50free_data = _50free_data[(_50free_data.T != 0.0).any()]
_50free_data = _50free_data.astype('float16')
_50free_data = _50free_data.resample('1M').mean()
_50free_data

,50 Free
Date,
2020-03-31,24.187500
2020-04-30,24.046875


# 100 Free

In [11]:
_100free_data = competition_data['100 Free'].to_frame()
_100free_data = _100free_data[(_100free_data.T != 0.0).any()]
_100free_data = _100free_data.astype('float16')
_100free_data = _100free_data.resample('1M').mean()
_100free_data

,100 Free
Date,
2020-03-31,52.59375
2020-04-30,51.81250
2020-05-31,NaN
2020-06-30,NaN
2020-07-31,53.40625


# 200 Free

In [12]:
_200free_data = competition_data['200 Free'].to_frame()
_200free_data = _200free_data[(_200free_data.T != 0.0).any()]
_200free_data = _200free_data.astype('float16')
_200free_data = _200free_data.resample('1M').mean()
_200free_data

,200 Free
Date,
2020-03-31,114.5000
2020-04-30,112.1875
2020-05-31,NaN
2020-06-30,NaN
2020-07-31,115.8750


# 400 Free

In [13]:
_400free_data = competition_data['400 Free'].to_frame()
_400free_data = _400free_data[(_400free_data.T != 0.0).any()]
_400free_data = _400free_data.astype('float16')
_400free_data = _400free_data.resample('1M').mean()
_400free_data

,400 Free
Date,
2020-04-30,247.625


# 800 Free

In [14]:
_800free_data = competition_data['800 Free'].to_frame()
_800free_data = _800free_data[(_800free_data.T != 0.0).any()]
_800free_data = _800free_data.astype('float16')
_800free_data = _800free_data.resample('1M').mean()
_800free_data

,800 Free
Date,


# 1500 Free

In [15]:
_1500free_data = competition_data['1500 Free'].to_frame()
_1500free_data = _1500free_data[(_1500free_data.T != 0.0).any()]
_1500free_data = _1500free_data.astype('float16')
_1500free_data = _1500free_data.resample('1M').mean()
_1500free_data

,1500 Free
Date,


# 50 Fly

In [16]:
_50fly_data = competition_data['50 Fly'].to_frame()
_50fly_data = _50fly_data[(_50fly_data.T != 0.0).any()]
_50fly_data = _50fly_data.astype('float16')
_50fly_data = _50fly_data.resample('1M').mean()
_50fly_data

,50 Fly
Date,
2020-03-31,28.0625


# 100 Fly

In [17]:
_100fly_data = competition_data['100 Fly'].to_frame()
_100fly_data = _100fly_data[(_100fly_data.T != 0.0).any()]
_100fly_data = _100fly_data.astype('float16')
_100fly_data = _100fly_data.resample('1M').mean()
_100fly_data

,100 Fly
Date,
2020-03-31,60.09375


# 200 Fly

In [18]:
_200fly_data = competition_data['200 Fly'].to_frame()
_200fly_data = _200fly_data[(_200fly_data.T != 0.0).any()]
_200fly_data = _200fly_data.astype('float16')
_200fly_data = _200fly_data.resample('1M').mean()
_200fly_data

,200 Fly
Date,


# 50 Back

In [19]:
_50back_data = competition_data['50 Back'].to_frame()
_50back_data = _50back_data[(_50back_data.T != 0.0).any()]
_50back_data = _50back_data.astype('float16')
_50back_data = _50back_data.resample('1M').mean()
_50back_data

,50 Back
Date,
2020-03-31,29.125000
2020-04-30,30.671875


# 100 Back

In [20]:
_100back_data = competition_data['100 Back'].to_frame()
_100back_data = _100back_data[(_100back_data.T != 0.0).any()]
_100back_data = _100back_data.astype('float16')
_100back_data = _100back_data.resample('1M').mean()
_100back_data

,100 Back
Date,


# 200 Back

In [21]:
_200back_data = competition_data['200 Back'].to_frame()
_200back_data = _200back_data[(_200back_data.T != 0.0).any()]
_200back_data = _200back_data.astype('float16')
_200back_data = _200back_data.resample('1M').mean()
_200back_data

,200 Back
Date,


# 50 Breast

In [22]:
_50breast_data = competition_data['50 Breast'].to_frame()
_50breast_data = _50breast_data[(_50breast_data.T != 0.0).any()]
_50breast_data = _50breast_data.astype('float16')
_50breast_data = _50breast_data.resample('1M').mean()
_50breast_data

,50 Breast
Date,
2020-03-31,30.765625
2020-04-30,31.625000


# 100 Breast

In [23]:
_100breast_data = competition_data['100 Breast'].to_frame()
_100breast_data = _100breast_data[(_100breast_data.T != 0.0).any()]
_100breast_data = _100breast_data.astype('float16')
_100breast_data = _100breast_data.resample('1M').mean()
_100breast_data

,100 Breast
Date,
2020-03-31,67.0625
2020-04-30,67.2500


# 200 Breast

In [24]:
_200breast_data = competition_data['200 Breast'].to_frame()
_200breast_data = _200breast_data[(_200breast_data.T != 0.0).any()]
_200breast_data = _200breast_data.astype('float16')
_200breast_data = _200breast_data.resample('1M').mean()
_200breast_data

,200 Breast
Date,


# 100IM

In [25]:
_100im_data = competition_data['100 IM'].to_frame()
_100im_data = _100im_data[(_100im_data.T != 0.0).any()]
_100im_data = _100im_data.astype('float16')
_100im_data = _100im_data.resample('1M').mean()
_100im_data

,100 IM
Date,


# 200 IM

In [26]:
_200im_data = competition_data['200 IM'].to_frame()
_200im_data = _200im_data[(_200im_data.T != 0.0).any()]
_200im_data = _200im_data.astype('float16')
_200im_data = _200im_data.resample('1M').mean()
_200im_data

,200 IM
Date,
2020-03-31,130.500
2020-04-30,128.625


# 400 IM

In [27]:
_400im_data = competition_data['400 IM'].to_frame()
_400im_data = _400im_data[(_400im_data.T != 0.0).any()]
_400im_data = _400im_data.astype('float16')
_400im_data = _400im_data.resample('1M').mean()
_400im_data

,400 IM
Date,


In [28]:
@app.callback(
    Output("competition-graph-id", "figure"),
    [
        Input("competition-avg-total-selector", "value"),
        Input("competition-category-selector", "value"),
    ],
)
def update_bar_chart(avg_or_total, categories):
    # filter by categories
    new_data = data[categories]
    # if new_data is in the form
    if isinstance(new_data, pd.Series):
        new_data = new_data.to_frame()

    # get avg or total data
    total_data = []
    if type(categories) == str:
        category_list = list(new_data[categories])
        total_value = sum(category_list)
        total_data.append(total_value)
    else:
        for category in categories:
            category_list = list(new_data[category])
            total_value = sum(category_list)
            total_data.append(total_value)
    
    if avg_or_total == 'avg':
        for i in range(len(total_data)):
            total_data[i] /= new_data.shape[0]
            
    bar_chart_figure = {
        "data": [
            {
                "x": categories,
                "y": total_data,
                "type": "bar",
                "name": "1 month data",
            },
        ],
        "layout": {
            "paper_bgcolor": graph_bgcolor,
            "plot_bgcolor": graph_bgcolor,
            "font": {
                "color": "#FFFFFF"
            }
        },
    }
    
    return bar_chart_figure

In [29]:
@app.callback(
    Output("categorical-bar-chart", "figure"),
    [
        Input("category-avg-total-selector", "value"),
        Input("category-category-selector", "value"),
        Input("category-date-picker", "start_date"),
        Input("category-date-picker", "end_date"),
    ],
)
def update_categorical_bar_chart(avg_or_total, categories, start_date, end_date):
    # filter by date
    date_mask = (
        (data.Date >= start_date)
        & (data.Date <= end_date)
    )
    cat_data = data.loc[date_mask, :]

    # filter by categories
    cat_data = cat_data[categories]
    # if new_data is in the form
    if isinstance(cat_data, pd.Series):
        cat_data = cat_data.to_frame()

    # get avg or total data
    total_data = []
    if type(categories) == str:
        category_list = list(cat_data[categories])
        total_value = sum(category_list)
        total_data.append(total_value)
    else:
        for category in categories:
            category_list = list(cat_data[category])
            total_value = sum(category_list)
            total_data.append(total_value)
    
    if avg_or_total == 'avg':
        for i in range(len(total_data)):
            total_data[i] /= cat_data.shape[0]
            
    bar_chart_figure = {
        "data": [
            {
                "x": categories,
                "y": total_data,
                "type": "bar",
                "name": "1 month data",
                "max-width": 50,
            },
        ],
        "layout": {
            "paper_bgcolor": graph_bgcolor,
            "plot_bgcolor": graph_bgcolor,
            "font": {
                "color": "#FFFFFF"
            }
        },
    }
    
    return bar_chart_figure

In [ ]:
if __name__ == "__main__":
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

 in production, use a production WSGI server like gunicorn instead.

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [24/Sep/2021 14:20:39] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/Sep/2021 14:20:39] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [24/Sep/2021 14:20:39] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [24/Sep/2021 14:20:39] "POST /_dash-update-component HTTP/1.1" 200 -
